# partlist-io2md

A tool to create a Markdown part list from a BOM (Bill Of Materials) extracted
from a project designed with [Stud.IO CAD](https://www.bricklink.com/v3/studio/download.page).
For each submodel in the project a dedicated list is created inside the markdown file.

> MIT License
> 
> Copyright &copy; 2022 by Alessandro Varesi
> 
> Permission is hereby granted, free of charge, to any person obtaining a copy
> of this software and associated documentation files (the "Software"), to deal
> in the Software without restriction, including without limitation the rights
> to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
> copies of the Software, and to permit persons to whom the Software is
> furnished to do so, subject to the following conditions:
>
> The above copyright notice and this permission notice shall be included in all
> copies or substantial portions of the Software.
>
> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
> IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
> FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
> AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
> LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
> OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
> SOFTWARE.
>

In [19]:
%%capture

import sys
# Install pip packages in current Jupyter kernel
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install urllib
!{sys.executable} -m pip install beautifulsoup4

## Choose the IO file name

In next code cell set the CSV filename (with path if needed)

In [20]:
iofile = "../../sets/10281/10281-bags.io"
colorsfile = "StudioColorDefinition.txt"
partsfile = "parts-cache.json" 

Now `Run all` cells of this notebook

In [21]:
import zipfile
import pandas as pd
from IPython.display import display, Markdown

with zipfile.ZipFile(iofile) as z:
    # print (z.infolist())

    with z.open('.info', pwd = b'soho0909') as f:
        pass
        # print (f.readlines())

    model_file = z.open('model.ldr',  pwd = b'soho0909')
    
    for l in model_file.readlines():
      print (l.strip().decode('utf-8-sig'))

    model_file.seek(0)

0 FILE 10281-bags
0 Untitled Model
0 Name:  10281-bags
0 Author:
0 CustomBrick
1 16 90.000000 0.000000 -380.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 1
1 16 480.000000 0.000000 -320.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 2
1 16 770.000000 0.000000 -90.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 3
1 16 1020.000000 0.000000 -170.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 4
1 16 110.000000 0.000000 -1100.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 5
1 16 740.000000 4.000000 -1120.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 bag 6
1 16 1120.000000 4.000000 -1000.000000 1.000000 0.000000 0.000000 0.000000 1.000000 0.000000 0.000000 0.000000 1.000000 unbagged
0 NOFILE
0 FILE Bag 1
0 Bag 1
0 Name:  Bag 1
0 

In [22]:
# Get color information from StudioColorDefinition.txt 
# ... this file is from Stud.io application
import math

colors = pd.read_csv(colorsfile, sep='\t')
# colors

def getColorInfo(color: int):
    """Get 'BL Color Code' and 'BL Color Name'

    ### Parameter
    color : int
        the LDraw color code

    ### Returns
    bl_color : dictionary
        BL Color Code : int
            the Bricklink color code (integer)
        BL Color Name : str 
            the Bricklinr color name (str)
    
    """

    # (colors['BL Color Code'] == colors['BL Color Code']) is atrick to check is not a NaN
    r = colors.loc[(colors['LDraw Color Code'] == color) & (colors['BL Color Code'] == colors['BL Color Code'])]

    return {
        'BL Color Code': int(r.iloc[0]['BL Color Code']),
        'BL Color Name': r.iloc[0]['BL Color Name']
    }

# colors = colors[['LDraw Color Code','BL Color Code','BL Color Name']]

print(getColorInfo(84))
print(getColorInfo(28))

{'BL Color Code': 150, 'BL Color Name': 'Medium Nougat'}
{'BL Color Code': 69, 'BL Color Name': 'Dark Tan'}


In [23]:
# Get various information from Bricklink site by getting info direct from html pages
# Not using Official API that need a AccessKey given only to registered sellers users.
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup


part_url = 'https://www.bricklink.com/v2/catalog/catalogitem.page?P={part}&idColor={blcolor}'

def getPartInfo(part : str, blcolor: int):
    req = Request(f'https://www.bricklink.com/v2/catalog/catalogitem.page?P={part}&idColor={blcolor}', headers={'User-Agent': 'Mozilla/5.0'})
    html = urlopen(req, timeout=10).read()
    soup = BeautifulSoup(html, 'html.parser')
    # Extract data from html
    description = soup.find('span', {'id': 'item-name-title'})
    description = description.string if description else ''
    weight = soup.find('span', {'id': 'item-weight-info'})
    weight = weight.string if weight else ''
    dimension = soup.find('span', {'id': 'dimSec'})
    dimension = dimension.string if dimension else ''
    image = f'https://img.bricklink.com/ItemImage/PN/{blcolor}/{part}.png'
    image = image if description else ''
    return { 
        'partname': part, 
        'description': description, 
        'weight': weight,
        'dimension': dimension,
        'image': image
    }

print(getPartInfo('32064', 69))

{'partname': '32064', 'description': 'Technic, Brick 1 x 2 with Axle Hole', 'weight': '0.88g', 'dimension': '1 x 2 x 1 in studs', 'image': 'https://img.bricklink.com/ItemImage/PN/69/32064.png'}


In [24]:
# Scan model file and fill the BOM dictionary structure
# part = { 
# }

import json

submodels = {}
try:
    parts = json.load( open (partsfile))
except IOError:
    parts = {}

class Model:
    def __init__(self, name: str):
        self.name = name
        self.BOM = {}
        self.submodels = {}

    def addSubmodel(self, sm_name: str):
        sm_id = sm_name.lower()
        if sm_id not in submodels:
            print (f"{self.name}.addSubmodel({sm_name})")
            submodels[sm_id] = Model(sm_name)
        self.submodels[sm_id] = submodels[sm_id]

    def addPart(self, part: str, color: int):
        part_id = f'{color}_{part}'
        #print (f"{self.name}.addPart({part_id}) ",end='') #DEBUG
        if part_id not in parts:
        #    print (f"LegoPart({part}, {color}) ",end='')
            parts[part_id] = LegoPart(part, color)
        if part_id in self.BOM:
            self.BOM[part_id]['qty'] += 1
        else:
            self.BOM[part_id]={'name': part, 'color': color, 'qty': 1}
        # print(f'{self.BOM}')

    def printBOM(self, isSubModel = False):
        if not isSubModel:
            print(f'MODEL : {self.name}')
            print('--------------------')
        else:
            print(f'    {self.name}')
            print('    ----------')
        for id, p in self.BOM.items():
            print (f'{p["name"]}({parts[id].colorname}): {p["qty"]} ')
        for sm in self.submodels.values():
            sm.printBOM(isSubModel = True)

class LegoPart:
    def __init__(self, partname: str, color: int):
        self.partname = partname.split('.')[0]
        self.color = color
        c = getColorInfo(color)
        self.blcolor = c['BL Color Code'] 
        self.colorname = c['BL Color Name']
    
    def getPartInfo(self):
        p = getPartInfo(self.partname, self.blcolor)
        self.description = p['description']
        self.weight = p['weight']
        self.dimension = p['dimension']
        self.image = p['image']
        self.unknow = True if (self.description == '') else False


l = model_file.readline().strip().decode('utf-8-sig').split(' ')

model = Model(' '.join(l[2:]))
# throw away next 4 lines
model_file.readline()
model_file.readline()
model_file.readline()

print (f'Creating BOM for model : {model.name}')

thisModel = model

for l in model_file.readlines():

    line = l.strip().decode('utf-8-sig').split(' ')
    # check for line type 
    if line[0] == '0':
        if line[1] == 'FILE':
            sm_name = " ".join(line[2:])
            sm_id = sm_name.lower()
            if sm_id not in submodels:
                submodels[sm_id] = Model(sm_name)
            # if model is created from a line "1" the name is lowercase
            # rewrite name with the one defined on FILE 
            submodels[sm_id].name = sm_name # rewrite name 
            thisModel = submodels[sm_id]
            #print (f'Create submodel : {thisModel.name}')
    elif line[0] == '1':
        color = int(line[1]) # color
        part = " ".join(line[14:]) # part
        if color == 16:    # this is a submodel in the BOM
            thisModel.addSubmodel(part)
        elif color == 24:
            pass
        else:
            thisModel.addPart(part, color)

# update parts cache file
#for p in parts.values():
#    print(f'{p.partname} - Color: {p.color} ({p.colorname})')
    
    

Creating BOM for model : 10281-bags
10281-bags.addSubmodel(bag 1)
10281-bags.addSubmodel(bag 2)
10281-bags.addSubmodel(bag 3)
10281-bags.addSubmodel(bag 4)
10281-bags.addSubmodel(bag 5)
10281-bags.addSubmodel(bag 6)
10281-bags.addSubmodel(unbagged)
Bag 1.addSubmodel(transparent baglet 1a)
Bag 2.addSubmodel(transperent baglet 2a)
Bag 3.addSubmodel(transparent baglet 3a)
Bag 4.addSubmodel(transparent baglet 4a)
Bag 5.addSubmodel(transparent baglet 5a)
Bag 5.addSubmodel(transparent baglet 5b)
Bag 6.addSubmodel(transparent baglet 6a)


In [25]:
model.printBOM()


MODEL : 10281-bags
--------------------
    Bag 1
    ----------
3036.dat(Dark Tan): 2 
2445.dat(Black): 4 
4477.dat(Dark Bluish Gray): 4 
87609.dat(Dark Bluish Gray): 6 
61485.dat(White): 1 
30414.dat(Light Bluish Gray): 10 
87079.dat(Black): 12 
15068.dat(Black): 24 
3623.dat(Light Bluish Gray): 4 
59895.dat(Black): 5 
96874.dat(Dark Turquoise): 1 
    Transparent baglet 1a
    ----------
3022.dat(Black): 4 
40145.dat(Black): 4 
60478.dat(Black): 4 
99206.dat(Light Bluish Gray): 2 
3005.dat(Light Bluish Gray): 4 
27263.dat(White): 4 
6141.dat(White): 9 
    Bag 2
    ----------
11213.dat(Dark Tan): 1 
3795.dat(Dark Brown): 2 
3460.dat(Dark Brown): 1 
3666.dat(Reddish Brown): 1 
30099.dat(Reddish Brown): 2 
65473.dat(Reddish Brown): 4 
3004.dat(Reddish Brown): 2 
88292.dat(Reddish Brown): 2 
60477.dat(Dark Brown): 4 
93606.dat(Dark Brown): 1 
3023.dat(Dark Brown): 4 
15068.dat(Reddish Brown): 3 
2420.dat(Reddish Brown): 2 
3021.dat(Black): 2 
99206.dat(Reddish Brown): 1 
30166.dat(Red

In [26]:
print('Getting parts info from Bricklink')
for p in parts.values():
    print(f"{p.partname} {p.colorname}({p.blcolor})",end='')
    p.getPartInfo()
    if p.unknow: 
        print(" !! UNKNOW !!")
    else:
        print(" DONE")

Getting parts info from Bricklink
3036 Dark Tan(69) DONE
2445 Black(11) DONE
4477 Dark Bluish Gray(85) DONE
87609 Dark Bluish Gray(85) DONE
61485 White(1) DONE
30414 Light Bluish Gray(86) DONE
87079 Black(11) DONE
15068 Black(11) DONE
3623 Light Bluish Gray(86) DONE
59895 Black(11) DONE
96874 Dark Turquoise(39) DONE
11213 Dark Tan(69) DONE
3795 Dark Brown(120) DONE
3460 Dark Brown(120) DONE
3666 Reddish Brown(88) DONE
30099 Reddish Brown(88) DONE
65473 Reddish Brown(88) DONE
3004 Reddish Brown(88) DONE
88292 Reddish Brown(88) DONE
60477 Dark Brown(120) DONE
93606 Dark Brown(120) DONE
3023 Dark Brown(120) DONE
15068 Reddish Brown(88) DONE
2420 Reddish Brown(88) DONE
3021 Black(11) DONE
99206 Reddish Brown(88) DONE
30166 Reddish Brown(88) DONE
35044 Reddish Brown(88) DONE
44728 Reddish Brown(88) DONE
41682 Reddish Brown(88) DONE
41769 Reddish Brown(88) DONE
41770 Reddish Brown(88) DONE
2450 Reddish Brown(88) DONE
99207 Black(11) DONE
99781 Dark Turquoise(39) DONE
32064a Dark Brown(120) !

In [27]:
# md = "Picture | Qty | Code | Description | Color |\n"
# md += "--------|----:|------|-------------|-------|\n"
# for i in df.itertuples():
#    item = i.BLItemNo
#    qty = i.Qty
#    description = i.PartName
#     color = i.ColorName
#     colorid = i.BLColorId
#     image = f"![{item}](https://img.bricklink.com/P/{colorid:.0f}/{item}.jpg)"
#     link = f"[{item}](https://www.bricklink.com/v2/catalog/catalogitem.page?P={item}&idColor={colorid:.0f})"

#     md += f"{image}| {qty:.0f} | {link} | {description} | {color}\n"

# display(Markdown(md))

In [28]:
# outmd = "## Here is your markdown block\n"
# outmd += "Copy next code block and paste in your _md_ file\n"
# outmd += "```(copy)\n"
# outmd += md
# outmd += "```\n"

# display(Markdown(outmd))